In [1]:
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", PROJECT_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [5]:
cifar10=keras.datasets.cifar10
(X_train_full,y_train_full),(X_test,y_test)=cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

## Model 1

In [9]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal")) 
    
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
#logs events for TensorBoard visualisation
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]


model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 7s 4ms/step - loss: 6.5104 - accuracy: 0.1668 - val_loss: 2.1583 - val_accuracy: 0.2280
Epoch 2/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0856 - accuracy: 0.2380 - val_loss: 2.1277 - val_accuracy: 0.2428
Epoch 3/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.9702 - accuracy: 0.2778 - val_loss: 1.9222 - val_accuracy: 0.3006
Epoch 4/100
1407/1407 [==============================] - 5s 3ms/step - loss: 1.8893 - accuracy: 0.3088 - val_loss: 1.9275 - val_accuracy: 0.2996
Epoch 5/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.8290 - accuracy: 0.3344 - val_loss: 1.8183 - val_accuracy: 0.3304
Epoch 6/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.7820 - accuracy: 0.3514 - val_loss: 1.7660 - val_accuracy: 0.3452
Epoch 7/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.7445 - accuracy: 0.3656 - val_loss: 1.7329 - val_ac

1407/1407 [==============================] - 7s 5ms/step - loss: 1.2347 - accuracy: 0.5563 - val_loss: 1.6207 - val_accuracy: 0.4510
Epoch 58/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.2275 - accuracy: 0.5594 - val_loss: 1.6138 - val_accuracy: 0.4422


In [10]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5444 - accuracy: 0.4536


[1.5443766117095947, 0.4535999894142151]

# Model 2: add BN

In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=20,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/20
1407/1407 [==============================] - 17s 9ms/step - loss: 1.8298 - accuracy: 0.3448 - val_loss: 1.6172 - val_accuracy: 0.4194
Epoch 2/20
1407/1407 [==============================] - 12s 8ms/step - loss: 1.6656 - accuracy: 0.4065 - val_loss: 1.6086 - val_accuracy: 0.4208
Epoch 3/20
1407/1407 [==============================] - 12s 9ms/step - loss: 1.5985 - accuracy: 0.4336 - val_loss: 1.5548 - val_accuracy: 0.4414
Epoch 4/20
1407/1407 [==============================] - 12s 8ms/step - loss: 1.5490 - accuracy: 0.4496 - val_loss: 1.5274 - val_accuracy: 0.4530
Epoch 5/20
1407/1407 [==============================] - 12s 8ms/step - loss: 1.5077 - accuracy: 0.4663 - val_loss: 1.4578 - val_accuracy: 0.4806
Epoch 6/20
1407/1407 [==============================] - 12s 8ms/step - loss: 1.4667 - accuracy: 0.4783 - val_loss: 1.5036 - val_accuracy: 0.4660
Epoch 7/20
1407/1407 [==============================] - 12s 8ms/step - loss: 1.4339 - accuracy: 0.4933 - val_loss: 1.4052 - val_ac

[1.31648588180542, 0.54339998960495]

## Model 3

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled , y_train , epochs=20,
          validation_data=(X_valid_scaled , y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled , y_valid)

Epoch 1/20
1407/1407 [==============================] - 10s 6ms/step - loss: 1.8843 - accuracy: 0.3258 - val_loss: 1.7458 - val_accuracy: 0.3816
Epoch 2/20
1407/1407 [==============================] - 8s 5ms/step - loss: 1.6678 - accuracy: 0.4097 - val_loss: 1.7530 - val_accuracy: 0.3940
Epoch 3/20
1407/1407 [==============================] - 8s 5ms/step - loss: 1.5679 - accuracy: 0.4490 - val_loss: 1.7080 - val_accuracy: 0.3878
Epoch 4/20
1407/1407 [==============================] - 8s 5ms/step - loss: 1.4965 - accuracy: 0.4724 - val_loss: 1.5670 - val_accuracy: 0.4524
Epoch 5/20
1407/1407 [==============================] - 8s 5ms/step - loss: 1.4377 - accuracy: 0.4947 - val_loss: 1.5662 - val_accuracy: 0.4504
Epoch 6/20
1407/1407 [==============================] - 8s 5ms/step - loss: 1.3826 - accuracy: 0.5151 - val_loss: 1.4900 - val_accuracy: 0.4872
Epoch 7/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3421 - accuracy: 0.5296 - val_loss: 1.4755 - val_accuracy

[1.4601657390594482, 0.4997999966144562]

## Model 4

In [13]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
model.add(keras.layers.AlphaDropout(rate=0.1))
#Alpha Dropout is a Dropout that keeps mean and variance of inputs to their original values, in order to ensure the self-normalizing property even after this dropout
#Since we are using SELU and not BN
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled , y_train , epochs=20,
          validation_data=(X_valid_scaled , y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled , y_valid)

Epoch 1/20
1407/1407 [==============================] - 9s 5ms/step - loss: 1.8877 - accuracy: 0.3311 - val_loss: 1.9051 - val_accuracy: 0.3428
Epoch 2/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6588 - accuracy: 0.4164 - val_loss: 1.6182 - val_accuracy: 0.4224
Epoch 3/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5633 - accuracy: 0.4490 - val_loss: 1.6211 - val_accuracy: 0.4302
Epoch 4/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4945 - accuracy: 0.4726 - val_loss: 1.5239 - val_accuracy: 0.4568
Epoch 5/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4418 - accuracy: 0.4955 - val_loss: 1.5497 - val_accuracy: 0.4724
Epoch 6/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3899 - accuracy: 0.5154 - val_loss: 1.5593 - val_accuracy: 0.4778
Epoch 7/20
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3441 - accuracy: 0.5277 - val_loss: 1.5660 - val_accuracy:

[1.4937437772750854, 0.5022000074386597]

In [ ]:
#Try MC dropout maybe...